<a href="https://colab.research.google.com/github/zaki18mi/inv/blob/main/Inv2_Zakaria_(Version_detect_Patterns).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Installing packages in Google drive




In [ ]:
'''
import os, sys
#to be able to interact with Google Drive's operating system
from google.colab import drive
#drive is a module that allows us use Python to interact with google drive
drive.mount('/content/gdrive')
#mounting google drive allows us to work with its contents
nb_path = '/content/notebooks'
os.symlink('/content/gdrive/My Drive/ColabNotebooks/InvestingProject/Libraries', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)
#The last three lines are what changes the path of the file




#!pip install --target=$nb_path ccxt pandas_market_calendars colorama investpy
'''


"\nimport os, sys\n#to be able to interact with Google Drive's operating system\nfrom google.colab import drive\n#drive is a module that allows us use Python to interact with google drive\ndrive.mount('/content/gdrive')\n#mounting google drive allows us to work with its contents\nnb_path = '/content/notebooks'\nos.symlink('/content/gdrive/My Drive/ColabNotebooks/InvestingProject/Libraries', nb_path)\nsys.path.insert(0, nb_path)  # or append(nb_path)\n#The last three lines are what changes the path of the file\n\n\n\n\n#!pip install --target=$nb_path ccxt pandas_market_calendars colorama investpy\n"

Import packages

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/My Drive/ColabNotebooks/InvestingProject/Libraries')

import ccxt
import datetime
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas_market_calendars as mcal
import plotly.express as px
from colorama import Fore, Back, Style
import yfinance as yf
import investpy
import requests
import statistics
import openpyxl


Mounted at /content/gdrive


############################  Load Data        ###############################

In [ ]:
def getDataFromYahoo(symbol, start, end, frequency):
    ###getting data from KKraken

    data = yf.download(symbol, start= start, end = end, interval=frequency)

    # Reset index to regular column and select desired columns
    df = data.reset_index()[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

    # Rename columns
    df = df.rename(columns={'Date': 'datetime', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close',
                            'Volume': 'volume'})
    # Print the first few rows of the data
    #print(data.head(10))

    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    return df








###################     Candle & Trend functions        ########################






In [ ]:

def getCandleBodySize(candle):
    return abs(candle['close']-candle['open'])

def getCandlesBodySizes(df)->list:
    ## this function will store all the sizes of candles in a list, so we can calculate median, mean, ...

    index_length = len(df.index)
    candleBodySizes = []
    for i in range(index_length):
        candleBodySizes.append(getCandleBodySize(df.iloc[i]))

    return candleBodySizes


def checkCandleBodySize(candle, size):
    # verify if candle body size is greater than a threshold
    return getCandleBodySize(candle)>=size

def checkCandlesBodySize(candles, size):
    ### verify if all candles have a body size greater then a threshold
    for candle in candles:
        if not checkCandleBodySize(candle, size):
            return False
    return True

def getCandleSize(candle):
    return abs(candle['high']-candle['low'])

def getCandlesSizes(df)->list:
    ## this function will store all the sizes of candles in a list, so we can calculate median, mean, ...

    index_length = len(df.index)
    candleSizes = []
    for i in range(index_length):
        candleSizes.append(getCandleSize(df.iloc[i]))

    return candleSizes

def getCandlesMean(candleSizes):
    ### return the mean from list of floats
    return statistics.mean(candleSizes)

def getCandlesQuantiles(candleSizes):

    return statistics.quantiles(candleSizes,  method='inclusive')



def getCandleUpperShaddow(candle):
    if bullish(candle):
        return candle['high']-candle['close']
    else:
        return candle['high']-candle['open']

def getCandleLowerShaddow(candle):
    if bullish(candle):
        return candle['open'] - candle['low']
    else:
        return candle['close']-candle['low']

def bullish(candle):
    return (candle['close']-candle['open']) >= 0








def getCandleType(candle, candleSizes, candleBodySizes, percentageShadowThreshold)->str:
    #candleSizeThreshold allow to detect if the candle is small

    if (getCandleSize(candle) == 0):
        return "empty"
    else:
        percentageBody = getCandleBodySize(candle)/getCandleSize(candle)  ### percentage of body to the total size of a candle
        percentageUpperShadow = getCandleUpperShaddow(candle)/getCandleSize(candle) ###% of upperShadow
        percentageLowerShadow = getCandleLowerShaddow(candle)/getCandleSize(candle) ###% of lowerShadow

        type = " (Bullish)" if bullish(candle) else " (Bearish)"  ###this indicade if a candle is bullish or bearish
        outlier = ""
        if getCandleSize(candle) < candleSizes[0]: # first quartile
            outlier = "(Small)"
        elif getCandleSize(candle) > candleSizes[3]: #third quartile
            outlier = "(Big)"



        if percentageBody <= 0.1:
            return type+"Doji"+outlier

        if percentageBody >= 0.95:
            return type+"Full"+outlier

        if percentageBody >= 0.6 and ((percentageUpperShadow-percentageLowerShadow) <= 0.1):
            return type+"Medium"+outlier

        if (percentageBody>0.1 and percentageBody <= 0.40) and percentageUpperShadow <= percentageShadowThreshold :
            return type+"Hammer"+outlier

        if (percentageBody>0.1 and percentageBody <= 0.40) and percentageLowerShadow <= percentageShadowThreshold :
            return type+"InvertedHammer"+outlier

        if (percentageBody>0.4 and percentageBody <= 0.6) and percentageUpperShadow <= percentageShadowThreshold  :
            return type+"ShavedHead"+outlier

        if (percentageBody>0.4 and percentageBody <= 0.6) and percentageLowerShadow <= percentageShadowThreshold  :
            return type+"ShavedBottom"+outlier

        return type+"None"+outlier


def getCandleTypePossibilities():
  candlesPossibleType = []
  for i in ['(Bullish)' , '(Bearish)']:
    for j in ['Doji',  'Full',  'Medium',  'Hammer',   'InvertedHammer',  'ShavedHead',    'ShavedBottom',  'None']:
      for k in ['','(Small)','(Big)' ]:
          possibility = i+j+k
          candlesPossibleType.append (possibility)
  return candlesPossibleType


def addCandleType(df, candleSizes, candleBodySizes, percentageShadowThreshold):
    ### for each candle add the the type: hammer, full, small, inverted Hammer ...
    index_length = len(df.index)
    df['candleType'] = [getCandleType(df.iloc[i], candleSizes, candleBodySizes, percentageShadowThreshold) for i in range(index_length)]





def checkTrend(candles:list, down:bool, param1:float, param2:float):
    ### candles: list of candles in order
    ### down: True for checking downtrend, False for checking up trends
    ### param1: the percentage of candles that have to be bearish
    ### param2: the hight between the opening of the first candle and the closing of the last one is high enough for a downtrend/uptrend

    ###check if 70% (param1) of the candles are in the same trend
    nbBullish = 0
    nbBearish = 0

    for candle in candles:
        #maxi, mini = max(maxi, candle["close"]), min(maxi, candle["close"])
        #mini = max(maxi, candle["open"]), min(mini, candle["open"])
        if candle["close"] < candle["open"]:
            nbBearish += 1
        else:
            nbBullish +=1

    if down: ### if we are looking for down trends
        if (nbBearish / len(candles) >= param1):
            mini1 = min(candles[0]["close"], candles[0]["open"])
            mini2 = min(candles[-1]["close"], candles[-1]["open"])
            ### check if the the hight between the opening of the first candle and the closing of the last one is high enough for a downtrend (param2 0.01
            dateTmp0 = candles[0]["datetime"]
            dateTmp1 = candles[-1]["datetime"]
            #if "2021-09-30T02:00:00" in str(dateTmp0):
            return ((mini1-mini2) >= param2)
    else: ### if we are looking for an upt rend
        if (nbBullish / len(candles) >= param1):
            maxi1 = max(candles[0]["close"], candles[0]["open"])
            maxi2 = max(candles[-1]["close"], candles[-1]["open"])
            return ((maxi2 - maxi1) >= param2)
    return False

Plots

In [ ]:
def plotCandlesticksSimple(df):


    # Create subplots and mention plot grid size
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True,
                        vertical_spacing=0.03, subplot_titles=('Air Liquid',))

    # Plot EUR/USD on 2nd row
    fig.add_trace(go.Candlestick(x=df["datetime"],  open=df["open"], high=df["high"], low=df["low"], close=df["close"],
                                 name="Air Liquide"), row=1, col=1)
    # Do not show OHLC's rangeslider plot
    fig.update(layout_xaxis_rangeslider_visible=False)

    # Customize x-axis to show only weekdays
    fig.update_xaxes(
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends
        ],
        tickformat='%d/%m/%Y',  # set tick format
    )

    fig.show()








def plotCandlesticks(df, patterns):
    # Convert timestamp to datetime
    #df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Create subplots and mention plot grid size
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.03, subplot_titles=('Euro/Usd', 'Volume'),
                        row_width=[0.2, 0.7])

    # Plot EUR/USD on 1st row
    fig.add_trace(go.Candlestick(x=df["datetime"], open=df["open"], high=df["high"], low=df["low"], close=df["close"], name="EUR/USD"),row=1, col=1 )

    # Bar trace for volumes on 2nd row without legend
    fig.add_trace(go.Bar(x=df["datetime"], y=df['volume'], showlegend=False), row=2, col=1)

    # Add annotations for bullish_engulfing signals
    annotations = []
    if "bearish_engulfing" in patterns:
        for i, row in df[df["bearish_engulfing"] == True].iterrows():
            annotations.append(dict(x=row["datetime"], y=row["high"], xref="x", yref="y",
                                    text="Bearish Engulfing", showarrow=True, arrowhead=2, ax=0, ay=-40))

    if "bullish_engulfing" in patterns:
        for i, row in df[df["bullish_engulfing"] == True].iterrows():
            annotations.append(dict(x=row["datetime"], y=row["low"], xref="x", yref="y",
                                    text="Bullish Engulfing", showarrow=True, arrowhead=2, ax=0, ay=+40))

    fig.layout.annotations = annotations

    # Add a reset axes button
    reset_axes_button = dict(
        type="buttons",
        showactive=False,
        buttons=[
            dict(
                label='Reset axes',
                method='relayout',
                args=['xaxis.autorange', 'yaxis.autorange']
            )
        ]
    )

    # Add a zoom in and zoom out button
    zoom_button = dict(
        type="buttons",
        showactive=False,
        buttons=[
            dict(
                label='Zoom In',
                method='relayout',
                args=[{'xaxis.autorange': False, 'yaxis.autorange': False},
                      {'xaxis.range': [df.datetime[0], df.datetime[round(len(df) / 2)]],
                       'yaxis.range': [min(df.low), max(df.high)]}]
            ),
            dict(
                label='Zoom Out',
                method='relayout',
                args=[{'xaxis.autorange': True, 'yaxis.autorange': True}]
            )
        ]
    )

    fig.update_layout(
        updatemenus=[reset_axes_button, zoom_button],
    )

    # Do not show OHLC's rangeslider plot
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.layout.yaxis2.showgrid = False
    fig.show()










def plotCandlesticksWithRSI(df, patterns):
    # Convert timestamp to datetime
    #df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Create subplots and mention plot grid size
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        vertical_spacing=0.03, subplot_titles=('Euro/Usd', 'Volume', 'RSI'),
                        row_width=[0.1, 0.2, 0.7])

    # Plot EUR/USD on 2nd row
    fig.add_trace(go.Candlestick(x=df["datetime"],  open=df["open"], high=df["high"], low=df["low"], close=df["close"],
                                 name="EUR/USD"), row=1, col=1)




    # Compute RSI with talib
    rsi_period = 14
    rsi_values = calculate_rsi(df['close'], rsi_period)

    # Add RSI trace on 1st row
    fig.add_trace(go.Scatter(x=df["datetime"], y=rsi_values, mode='lines', name='RSI'), row=2, col=1)
    # add horizontal lines at 30% and 70% levels
    fig.add_shape(type='line', x0=df['datetime'].iloc[0], y0=30, x1=df['datetime'].iloc[-1], y1=30,
                  line=dict(color='orange', width=1, dash='dash'), row=2, col=1)
    fig.add_shape(type='line', x0=df['datetime'].iloc[0], y0=70, x1=df['datetime'].iloc[-1], y1=70,
                  line=dict(color='orange', width=1, dash='dash'), row=2, col=1)

    # Bar trace for volumes on 3rd row without legend
    fig.add_trace(go.Bar(x=df["datetime"], y=df['volume'], showlegend=False), row=3, col=1)

    # Add annotations for bullish_engulfing signals
    annotations = []
    if "bearish_engulfing" in patterns:
        for i, row in df[df["bearish_engulfing"] == True].iterrows():
            annotations.append(dict(x=row["datetime"], y=row["high"], xref="x", yref="y",
                                    text="Bearish Engulfing", showarrow=True, arrowhead=2, ax=0, ay=-40))

    if "bullish_engulfing" in patterns:
        for i, row in df[df["bullish_engulfing"] == True].iterrows():
            annotations.append(dict(x=row["datetime"], y=row["low"], xref="x", yref="y",
                                    text="Bullish Engulfing", showarrow=True, arrowhead=2, ax=0, ay=+40))

    fig.layout.annotations = annotations






    # Add a reset axes button
    reset_axes_button = dict(
        type="buttons",
        showactive=False,
        buttons=[
            dict(
                label='Reset axes',
                method='relayout',
                args=['xaxis.autorange', 'yaxis.autorange']
            )
        ]
    )

    # Add a zoom in and zoom out button
    zoom_button = dict(
        type="buttons",
        showactive=False,
        buttons=[
            dict(
                label='Zoom In',
                method='relayout',
                args=[{'xaxis.autorange': False, 'yaxis.autorange': False},
                      {'xaxis.range': [df.datetime[0], df.datetime[round(len(df) / 2)]],
                       'yaxis.range': [min(df.low), max(df.high)]}]
            ),
            dict(
                label='Zoom Out',
                method='relayout',
                args=[{'xaxis.autorange': True, 'yaxis.autorange': True}]
            )
        ]
    )

    fig.update_layout(
        updatemenus=[reset_axes_button, zoom_button],
    )

    # Customize x-axis to show only weekdays
    fig.update_xaxes(
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends
        ],
        tickformat='%d/%m/%Y',  # set tick format
    )

    # Do not show OHLC's rangeslider plot
    fig.update(layout_xaxis_rangeslider_visible=False)
    #fig.update_xaxes(type='category')
    fig.layout.yaxis2.showgrid = False
    fig.show()













def calculate_rsi(prices, n=14):
    ###this function is the same as addRSI, but it is used for the plot

    # First calculate the difference in price from previous day
    delta = prices.diff()

    # Make a copy of the data to preserve the original data
    d1 = delta.copy()
    d2 = delta.copy()

    # Replace positive deltas with 0 and negative deltas with absolute value
    d1[d1 < 0] = 0

    # Replace positive deltas with 0 and negative deltas with absolute value
    d2[d2 > 0] = 0
    d2[d2 < 0] = -d2

    # Calculate the rolling mean of gain and loss using the window size n
    roll_up = d1.rolling(window=n, center=False).mean()
    roll_down = d2.rolling(window=n, center=False).mean()


    # Calculate RSI based on the rolling mean of gain and loss
    rs = roll_up / roll_down
    rsi = 100 - (100 / (1 + rs))

    # Return the RSI value
    return rsi




def addRSI(df, period=14):
    # First calculate the difference in price from previous day
    delta = df["close"].diff()

    # Make a copy of the data to preserve the original data
    d1 = delta.copy()
    d2 = delta.copy()

    # Replace positive deltas with 0 and negative deltas with absolute value
    d1[d1 < 0] = 0

    # Replace positive deltas with 0 and negative deltas with absolute value
    d2[d2 > 0] = 0
    d2[d2 < 0] = -d2

    # Calculate the rolling mean of gain and loss using the window size n
    roll_up = d1.rolling(window=period, center=False).mean()
    roll_down = d2.rolling(window=period, center=False).mean()


    # Calculate RSI based on the rolling mean of gain and loss
    rs = roll_up / roll_down
    rsi = 100 - (100 / (1 + rs))

    # Return the RSI value
    df["RSI"]=rsi



#############################       Patterns        ######################

In [ ]:
def detect_pattern_engulfing(df, trendHeight, nbCandlesTrend, percentageCandleTrend):

    bullish_engulfing = []
    # first get the index length of the dataframe
    index_length = len(df.index)

    # initialize the new column with False values
    df['bullish_engulfing'] = [False for i in range(index_length)]
    df['bearish_engulfing'] = [False for i in range(index_length)]


    # loop through the dataframe rows to find and mark the bullish engulfing patterns
    for i in range(nbCandlesTrend,index_length - 1):

        if checkCandlesBodySize([df.iloc[i-1],df.iloc[i]],trendHeight):
            concernedCandle = df.iloc[i]
            previousCandle = df.iloc[i-1]
            previousCandleBodySize = getCandleBodySize(previousCandle)
            concernedCandleBodySize = getCandleBodySize(concernedCandle)
            ### if concerned candle is bulish and previous one is bearish
            ### if concerned candle close above the previous one
            ### if concerned candle do not open to far from the closing of the previous candle
            ### if body of concerned candle is 1.2 higher than body of previous candle
            if(bullish(concernedCandle) and not bullish(previousCandle)) \
                    and (previousCandle['open'] < concernedCandle['close']) \
                    and ((previousCandle['close']+(previousCandleBodySize/3))  > concernedCandle['open']) \
                    and (concernedCandleBodySize  >= (1.2 * previousCandleBodySize)):
                    #and (previousCandle['close'] >= concernedCandle['open'])

                ### if downtrend is True
                candles = [] ### nb candles before the concerned one
                for j in range (nbCandlesTrend,0,-1):
                    candles.append(df.iloc[i-j])

                if checkTrend(candles, True, percentageCandleTrend, trendHeight):
                #if (df.loc[i, 'close'] < df.loc[i - 1, 'close']) & (df.loc[i - 1, 'close'] < df.loc[i - 2, 'close']) & (df.loc[i - 2, 'close'] < df.loc[i - 3, 'close']):
                    downtrend = True
                    df.loc[df.index[i], 'bullish_engulfing'] = True

            ### if concerned candle is bulish and previous one is bearish
            ### if concerned candle close above the previous one
            ### if body of concerned candle is 1.2 higher than body of previous candle
            if (not bullish(concernedCandle) and bullish(previousCandle)) \
                    and (previousCandle['open'] > concernedCandle['close']) \
                    and ((previousCandle['close'] - (previousCandleBodySize / 3)) < concernedCandle['open']) \
                    and (getCandleBodySize(concernedCandle) >= (1.2 * getCandleBodySize(previousCandle))):
                ### if uptrend is True
                candles = []  ### nb candles before the concerned one
                #print("checking trend up of candle ", datetime.datetime.fromtimestamp(concernedCandle["timestamp"] / 1000.0).isoformat())
                for j in range(nbCandlesTrend, 0, -1):
                    candles.append(df.iloc[i - j])
                if checkTrend(candles, False, percentageCandleTrend, trendHeight):
                    uptrend = True
                    df.loc[df.index[i], 'bearish_engulfing'] = True



    return df


check sucess of all candles of DF

In [ ]:
def check_success_pattern_engulfing(df, nbCandles, gain, showDetails:bool)->[list]:
    ###this function calculate the sucess/failure of engulfin pattern for a dataframe
    ###It returns a dictionnary of these elements : "nbSuccessBullish" "nbBullishEngulfing" "gainBullish" "riskBullish" "nbSuccessBearish" "nbBearishEngulfing" "gainBearish" "riskBearish" "data"


    index_length = len(df.index)

    nbBearishEngulfing = 0 #number of  times that the pattern appears
    nbBullishEngulfing = 0

    nbSuccessBullish = 0 #number of times the pattern worked !
    nbSuccessBearish = 0

    gainLossBullish = 0 #gain/loss obtained
    gainLossBearish = 0

    riskBearish = 0 #risk taken
    riskBullish = 0

    data = []

    for i in range (1,index_length-nbCandles): #for each candle
        concernedCandle = df.iloc[i]
        dateConcernedCandle = concernedCandle["datetime"]
        previousCandle = df.iloc[i - 1]

        ###get the next n candles of a candle
        nextCandles = []
        for j in range (1,nbCandles+1):
            nextCandles.append(df.iloc[i+j])

        ### if the candle is engulfing patter
        if concernedCandle['bearish_engulfing']:

            nbBearishEngulfing += 1
            result = check_success_engulfing_next_candles(concernedCandle, previousCandle, nextCandles, "bearish_engulfing", gain) ###get informations of results
            if result[0]: #if success
                nbSuccessBearish += 1
            gainLossBearish += result[1]
            lossStop = min(concernedCandle["high"], previousCandle["high"])
            tmpRisk = abs(concernedCandle['close']-lossStop)
            riskBearish += tmpRisk

            #save data of one candle that is engulfing
            rowData = {"previousCandle":previousCandle,"concernedCandle":concernedCandle,"dateConcernedCandle":dateConcernedCandle, "gains":result[1], "risk": tmpRisk, "type":"bearish", "result":"success" if result[0] else "failure"}
            data.append(rowData)

            if showDetails:#for printing/testing
                print (Fore.RED+("+" if result[0] else Back.LIGHTCYAN_EX+"-"), " (",round(result[1],4),") bearish_engulfing: ",concernedCandle['candleType'], "  ",  dateConcernedCandle, " (RSI=",concernedCandle["RSI"] if concernedCandle["RSI"]!= None else "---" ,"%) Risk =",tmpRisk, " gain/loss=",result[1], Style.RESET_ALL)

        if concernedCandle['bullish_engulfing']:
            nbBullishEngulfing += 1
            result = check_success_engulfing_next_candles(concernedCandle, previousCandle, nextCandles,"bullish_engulfing", gain)
            if result[0]:
                nbSuccessBullish += 1
            gainLossBullish += result[1]

            lossStop = max(concernedCandle["low"], previousCandle["low"])
            tmpRisk = abs(concernedCandle['close']-lossStop)
            riskBullish += tmpRisk
            #print ("candle ", concernedCandle["datetime"], "risk=", concernedCandle['close'], "-", lossStop )#################zzzzzzzzzzzzzzzzzzzzzzz

            rowData = {"previousCandle":previousCandle,"concernedCandle":concernedCandle,"dateConcernedCandle":dateConcernedCandle, "gains":result[1], "risk": tmpRisk, "type":"bullish", "result":"success" if result[0] else "failure"}
            data.append(rowData)

            if showDetails:
                print(Fore.GREEN+ ("+" if result[0] else Back.LIGHTWHITE_EX+"-"), " (",round(result[1],4),") Bullish_engulfing: ", concernedCandle['candleType'], "  ", dateConcernedCandle, " (RSI=",round(concernedCandle["RSI"]),"%) Risk =",tmpRisk, " gain/loss=",result[1], Style.RESET_ALL)

    if showDetails:
        print()
        print ("BullishEngulfing = (",nbSuccessBullish,"/",nbBullishEngulfing,") gainLoss =",gainLossBullish,   "BearishEngulfing = (",nbSuccessBearish,"/",nbBearishEngulfing,") gainLoss = ",gainLossBearish )


    return {"nbSuccessBullish":nbSuccessBullish, "nbBullishEngulfing":nbBullishEngulfing, "gainBullish":gainLossBullish, "riskBullish":riskBullish, "nbSuccessBearish":nbSuccessBearish, "nbBearishEngulfing":nbBearishEngulfing, "gainBearish":gainLossBearish, "riskBearish":riskBearish, "data":data}


Check success of one candle

In [ ]:
from numpy.core.multiarray import result_type
def check_success_engulfing_next_candles(concernedCandle, previousCandle, nextCandles:list, engolfingType, gain)->(bool, float):

    #this strategy check if the next candles we hit the stop loss threshold, or we hit the stop gain threshold
    # if we do not hit one of these two thresholds in the next candles, we consider it as a loss or gain depending on the close of the candle!

    if engolfingType not in ["bullish_engulfing", "bearish_engulfing"]:
        raise ValueError ("Ayyyy this is not engulfing pattern Zak...")

    if engolfingType == "bullish_engulfing":
        lossStop = max(concernedCandle["low"],previousCandle["low"])
        gainStop = concernedCandle["close"]+gain

        #if str(concernedCandle["datetime"]) == "2021-03-18 00:00:00":
        #  print ("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz", lossStop, gainStop,"(",concernedCandle["close"],"-",gain,")")


        if (nextCandles[0]["open"]<lossStop): #if the oppening of a trade is in a jump
            return (False,0)

        if gain != -1: #gain != -1 means that we close the trade after the end of the candle (close)
            for nextCandle in nextCandles:
                #print ("  testing candle ",datetime.datetime.fromtimestamp(nextCandle["timestamp"] / 1000.0).isoformat(), " for candle ",datetime.datetime.fromtimestamp(concernedCandle["timestamp"] / 1000.0).isoformat()+ "loss=",concernedCandle["close"],"-",lossStop, "=",(concernedCandle["close"]-lossStop))
                if nextCandle["low"] <= lossStop:
                    #print ("  testing candle ",datetime.datetime.fromtimestamp(nextCandle["timestamp"] / 1000.0).isoformat(), " for candle ",datetime.datetime.fromtimestamp(concernedCandle["timestamp"] / 1000.0).isoformat()+ "loss=",lossStop,"-",concernedCandle["close"], "=",(lossStop-concernedCandle["close"]))
                    return (False, (lossStop - concernedCandle["close"]))
                if nextCandle["high"] >= gainStop:
                    return (True,gain)

        result = (nextCandles[-1]["close"]-concernedCandle["close"])
        #if str(concernedCandle["datetime"]) == "2021-03-18 00:00:00":
        #  print (result,"(",nextCandles[-1]["close"],"-",concernedCandle["close"])
        return (True if result>0 else False, result)



    else: ### bearish_engulfing
        lossStop = min(concernedCandle["high"], previousCandle["high"])
        gainStop = concernedCandle["close"] - gain

        #if str(concernedCandle["datetime"]) == "2018-10-22 00:00:00":
        #  print ("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz", lossStop, gainStop,"(",concernedCandle["close"],"-",gain,")")

        if (nextCandles[0]["open"]>lossStop): #if the oppening of a trade is in a jump
           return (False,0)

        if gain != -1: #gain != -1 means that we close the trade after the end of the candle (close)
            for nextCandle in nextCandles:
                #print ("  testing candle ",datetime.datetime.fromtimestamp(nextCandle["timestamp"] / 1000.0).isoformat(), " for candle ",datetime.datetime.fromtimestamp(concernedCandle["timestamp"] / 1000.0).isoformat()+ "loss=",lossStop,"-",concernedCandle["close"], "=",(lossStop-concernedCandle["close"]))
                if nextCandle["high"] > lossStop:
                    return (False,(concernedCandle["close"] - lossStop))
                if nextCandle["low"] <= gainStop:
                    return (True, gain)
        result = (concernedCandle["close"] - nextCandles[-1]["close"])
        #if str(concernedCandle["datetime"]) == "2018-10-22 00:00:00":
        #   print ("okkk", result)
        return (True if result>0 else False, result)


##############################   Parameters of Engulfing      ################

In [ ]:
def get_best_engulfing_parameters(df, nbSuccessLowerBound, pattern, message):
    ###nbSuccessLowerBound represent the minimum number of success that we want to have for the patern

    #This function enemurate all the parameters (gainStop, nbCandles, ... ) and return the best ones for the engulfing patterns
    #it means that it returns the parameters that worked better: We have three strategies to say that the parameters worked better
    #Strategy 1: (parameters1) it check the number (percentage) of time the pattern worked without looking at the gains/stops
    #Strategy 2: (parameters2) it gives the parameters that give the best gains without looking at the number of time the pattern worked or not
    #Strategy 3: (parameters3) a mixe between strategy 1 and 2  ?


    # parameters of candles : we look at the candle sizes, and use these informations for the parameters of trend ....
    candleSizes = getCandlesSizes(df)
    candleBodySizes = getCandlesBodySizes(df)
    candlesQuantiles = getCandlesQuantiles(candleBodySizes)


    ### parameters of a trend
    trendHeightPossibilities = candlesQuantiles#param1
    nbCandlesTrendPossibilities = [3, 4, 5] #param2
    percentageCandleTrendPossibilities = [0.7, 0.8, 0.9, 1] #param3



    ### parameters for type of candle
    #percentageShadowThresholdPossibilities = [0.5, 0.1, 0.15, 0.2]
    #candleSizeThresholdPossibilities = [candlesQuantiles[0], getCandlesMean(candleSizes)/2]

    ### parameters for checking engulfing pattern sucess
    nbCandlesPossibilities = [1, 2, 3, 4, 5]  #param4   #wait 5 days if we have daily candles, or 5 weeks if we have weekly candles
    gainPossibilties = [-1, getCandlesMean(candleSizes), candlesQuantiles[1], candlesQuantiles[2], getCandlesMean(candleSizes)*2, candlesQuantiles[1]*2] #param5



    ### initiatize best parameters
    bestBullishParameters1 = {}  ### this contains best parameters when looking at the number of times  (percentage) the patter works without considering the gains
    bestBearishParameters1 = {}
    bestBullishParameters2 = {}  ### this contains best parameters when looking at the maximum gains without considering the number of times the patter works
    bestBearishParameters2 = {}
    bestBullishParameters3 = {}  ### this contains best parameters when looking at the number of times the patter works and considering the gains as well
    bestBearishParameters3 = {}



    bestBullishGain = 0
    bestBearishGain = 0
    bestNbBullishGain = 0
    bestNbBearishGain = 0
    bestNbBullishTotal = 1
    bestNbBearishTotal = 1

    #USE PARAMETER3 VARIABLES INDEPENDENTLY
    bestBullishGainParam3 = 0
    bestBearishGainParam3 = 0
    bestNbBullishGainParam3 = 0
    bestNbBearishGainParam3 = 0

    nb =0
    nbPossibilities = len(trendHeightPossibilities) * len(nbCandlesTrendPossibilities) * len(percentageCandleTrendPossibilities) * len(nbCandlesPossibilities) * len (gainPossibilties)

    for trendHeight in trendHeightPossibilities: #param1
        for nbCandlesTrend in nbCandlesTrendPossibilities:  #param2
            for percentageCandleTrend in percentageCandleTrendPossibilities: #param3
                for nbCandles in nbCandlesPossibilities: #param4
                    for gainStop in gainPossibilties: #param5
                        nb += 1
                        # Detect patterns
                        df = detect_pattern_engulfing(df, trendHeight, nbCandlesTrend, percentageCandleTrend)
                        tmpResult = check_success_pattern_engulfing(df, nbCandles, gainStop, False)
                        #tmpResult has : {"nbSuccessBullish", "nbBullishEngulfing", "gainBullish", "riskBullish", "nbSuccessBearish", "nbBearishEngulfing", "gainBearish", "riskBearish", "data"}

                        #tmpNbGain = tmpResult["nbSuccessBullish"]+tmpResult[4]
                        tmpNbTotal = tmpResult["nbBullishEngulfing"]+tmpResult["nbBearishEngulfing"]

                        tmpNbBullishGain = tmpResult["nbSuccessBullish"]
                        tmpNbBullishTotal = tmpResult["nbBullishEngulfing"]
                        tmpBullishGain = tmpResult["gainBullish"]
                        tmpBullishRisk = tmpResult["riskBullish"]

                        tmpNbBearishGain = tmpResult["nbSuccessBearish"]
                        tmpNbBearishTotal = tmpResult["nbBearishEngulfing"]
                        tmpBearishGain = tmpResult["gainBearish"]
                        tmpBearishRisk = tmpResult["riskBearish"]
                        #percentageNbGain = 0 if nbTotal ==0 else (nbGains/nbTotal)

                        tmpData = tmpResult["data"]

                        changeBestBullishParameters1 = False
                        changeBestBearishParameters1 = False
                        changeBestBullishParameters2 = False
                        changeBestBearishParameters2 = False
                        changeBestBullishParameters3 = False
                        changeBestBearishParameters3 = False
                        style = Fore.WHITE

                        #Bullish cases
                        if tmpNbBullishTotal <= nbSuccessLowerBound or bestNbBullishGain <= nbSuccessLowerBound:  ### if we compare small numbers we do not use %
                            if tmpNbBullishGain > bestNbBullishGain:
                                changeBestBullishParameters1 = True
                        else:
                            if (tmpNbBullishGain/tmpNbBullishTotal) > (bestNbBullishGain/bestNbBullishTotal):
                                changeBestBullishParameters1 = True

                        if tmpBullishGain > bestBullishGain:
                            changeBestBullishParameters2 = True

                        if (tmpNbBullishGain >= bestNbBullishGainParam3) and (tmpBullishGain > bestBullishGainParam3):
                            changeBestBullishParameters3 = True


                        if changeBestBullishParameters1:
                            bestNbBullishGain = tmpNbBullishGain
                            bestNbBullishTotal = tmpNbBullishTotal
                            bestBullishParameters1 = {"trendHeight":trendHeight, "nbCandlesTrend":nbCandlesTrend, "percentageCandleTrend":percentageCandleTrend, "nbCandles":nbCandles, "gainStop":gainStop, "statBullish":str(tmpNbBullishGain)+"/"+str(tmpNbBullishTotal), "statBearish":str(tmpNbBearishGain)+"/"+str(tmpNbBearishTotal), "gains":tmpBullishGain, "risk":tmpBullishRisk, "data":tmpData }
                            style = Fore.GREEN

                        if changeBestBullishParameters2:
                            bestBullishGain = tmpBullishGain
                            bestBullishParameters2 = {"trendHeight":trendHeight, "nbCandlesTrend":nbCandlesTrend, "percentageCandleTrend":percentageCandleTrend, "nbCandles":nbCandles, "gainStop":gainStop, "statBullish":str(tmpNbBullishGain)+"/"+str(tmpNbBullishTotal), "statBearish":str(tmpNbBearishGain)+"/"+str(tmpNbBearishTotal), "gains":tmpBullishGain, "risk":tmpBullishRisk, "data":tmpData }
                            style = Fore.GREEN

                        if changeBestBullishParameters3:
                            bestNbBullishGainParam3 = tmpNbBullishGain
                            bestBullishGainParam3 = tmpBullishGain
                            bestBullishParameters3 = {"trendHeight":trendHeight, "nbCandlesTrend":nbCandlesTrend, "percentageCandleTrend":percentageCandleTrend, "nbCandles":nbCandles, "gainStop":gainStop, "statBullish":str(tmpNbBullishGain)+"/"+str(tmpNbBullishTotal), "statBearish":str(tmpNbBearishGain)+"/"+str(tmpNbBearishTotal), "gains":tmpBullishGain, "risk":tmpBullishRisk, "data":tmpData }
                            style = Fore.YELLOW


                        #Bearish cases
                        if tmpNbBearishTotal <= nbSuccessLowerBound or bestNbBearishGain <= nbSuccessLowerBound:  ### if we compare small numbers we do not use %
                            if tmpNbBearishGain > bestNbBearishGain:
                                changeBestBearishParameters1 = True
                        else:
                            if (tmpNbBearishGain/tmpNbBearishTotal) > (bestNbBearishGain/bestNbBearishTotal):
                                changeBestBearishParameters1 = True

                        if tmpBearishGain > bestBearishGain:
                            changeBestBearishParameters2 = True

                        if (tmpNbBearishGain >= bestNbBearishGainParam3) and (tmpBearishGain > bestBearishGainParam3):
                            changeBestBearishParameters3 = True


                        if changeBestBearishParameters1:
                            bestNbBearishGain = tmpNbBearishGain
                            bestNbBearishTotal = tmpNbBearishTotal
                            bestBearishParameters1 = {"trendHeight":trendHeight,"nbCandlesTrend": nbCandlesTrend, "percentageCandleTrend":percentageCandleTrend,"nbCandles": nbCandles, "gainStop":gainStop, "statBullish":str(tmpNbBullishGain)+"/"+str(tmpNbBullishTotal), "statBearish":str(tmpNbBearishGain)+"/"+str(tmpNbBearishTotal), "gains": tmpBearishGain, "risk": tmpBearishRisk, "data":tmpData }
                            style = Fore.GREEN

                        if changeBestBearishParameters2:
                            bestBearishGain = tmpBearishGain
                            bestBearishParameters2 = {"trendHeight":trendHeight, "nbCandlesTrend":nbCandlesTrend, "percentageCandleTrend":percentageCandleTrend,"nbCandles": nbCandles, "gainStop":gainStop, "statBullish":str(tmpNbBullishGain)+"/"+str(tmpNbBullishTotal), "statBearish":str(tmpNbBearishGain)+"/"+str(tmpNbBearishTotal), "gains":tmpBearishGain, "risk":tmpBearishRisk, "data":tmpData }
                            style = Fore.GREEN

                        if changeBestBearishParameters3:
                            bestNbBearishGainParam3 = tmpNbBearishGain
                            bestBearishGainParam3 = tmpBearishGain
                            bestBearishParameters3 = {"trendHeight":trendHeight, "nbCandlesTrend":nbCandlesTrend, "percentageCandleTrend":percentageCandleTrend,"nbCandles": nbCandles, "gainStop":gainStop, "statBullish":str(tmpNbBullishGain)+"/"+str(tmpNbBullishTotal), "statBearish":str(tmpNbBearishGain)+"/"+str(tmpNbBearishTotal), "gains":tmpBearishGain, "risk":tmpBearishRisk, "data":tmpData }
                            style = Fore.YELLOW


                        print(nb, '/', nbPossibilities)

                        if message and tmpResult != {}:
                            print (style+ "trendHeight=", round(trendHeight,2), "nbCandlesTrend=", nbCandlesTrend," percentageCandleTrend=", percentageCandleTrend, "nbCandles=", nbCandles,"gain=", round(gainStop,2),
                                Fore.GREEN if changeBestBullishParameters1 else Fore.WHITE, "[",tmpNbBullishGain,'/',tmpNbBullishTotal, Fore.WHITE, " bullish",
                                Fore.GREEN if changeBestBullishParameters2 else Fore.WHITE ,"(",round(tmpBullishGain,2),")", Fore.WHITE,  "and ",
                                Fore.GREEN if changeBestBearishParameters1 else Fore.WHITE ,  tmpNbBearishGain,'/',tmpNbBearishTotal, Fore.WHITE, " bearish",
                                Fore.GREEN if changeBestBearishParameters2 else Fore.WHITE ,"(",round(tmpBearishGain,2),")]",
                                Fore.YELLOW if changeBestBullishParameters3 else Fore.WHITE ,  "Bullish param3",  Fore.YELLOW if changeBestBearishParameters3 else Fore.WHITE ,  "Bearish param3",
                                Style.RESET_ALL)

                        #if not changeBestBullishParameters1 and not changeBestBullishParameters2 and not changeBestBearishParameters1 and not changeBestBearishParameters2:
                        #    nb += len(gainPossibilties[gainStop:])
                        #    break


    '''
    try:
        print ("\n\n best result1 Bullish:   [",bestBullishResult1[0],'/',bestBullishResult1[1]," bullish (",round(bestBullishResult1[2],2),") and ", bestBullishResult1[3],'/',bestBullishResult1[4]," bearish (",round(bestBullishResult1[5],2),")]")
        print ("\n\n best result2 Bullish: Gain = ",round(bestBullishGain,2), "[",bestBullishResult2[0],'/',bestBullishResult2[1]," bullish (",round(bestBullishResult2[2],2),") and ", bestBullishResult2[3],'/',bestBullishResult2[4]," bearish (",round(bestBullishResult2[5],2),")]")

        print ("\n\n best result1 Bearish:   [",bestBearishResult1[0],'/',bestBearishResult1[1]," bullish (",round(bestBearishResult1[2],2),") and ", bestBearishResult1[3],'/',bestBearishResult1[4]," bearish (",round(bestBearishResult1[5],2),")]")
        print ("\n\n best result2 Bearish: Gain = ",round(bestBearishGain,2), "[",bestBearishResult2[0],'/',bestBearishResult2[1]," bullish (",round(bestBearishResult2[2],2),") and ", bestBearishResult2[3],'/',bestBearishResult2[4]," bearish (",round(bestBearishResult2[5],2),")]")

        print ("\nbest Bullish parameters1:  trendHeight=", bestBullishParameters1[0], "nbCandlesTrend=", bestBullishParameters1[1]," percentageCandleTrend=", bestBullishParameters1[2],
              "nbCandles=", bestBullishParameters1[3],"gain-loss-out=", bestBullishParameters1[4],"  resultats=",round((bestNbBullishGain/bestNbBullishTotal),2) )
        print ("\nbest Bullish parameters2:  trendHeight=", bestBullishParameters2[0], "nbCandlesTrend=", bestBullishParameters2[1]," percentageCandleTrend=", bestBullishParameters2[2],
              "nbCandles=", bestBullishParameters2[3],"gain-loss-out=", bestBullishParameters2[4], " resultats=",round(bestBullishGain,2) )

        print ("\nbest Bearish parameters1:  trendHeight=", bestBearishParameters1[0], "nbCandlesTrend=", bestBearishParameters1[1]," percentageCandleTrend=", bestBearishParameters1[2],
              "nbCandles=", bestBearishParameters1[3],"gain-loss-out=", bestBearishParameters1[4],"  resultats=",round((bestNbBearishGain/bestNbBearishTotal),2) )
        print ("\nbest Bearish parameters2:  trendHeight=", bestBearishParameters2[0], "nbCandlesTrend=", bestBearishParameters2[1]," percentageCandleTrend=", bestBearishParameters2[2],
              "nbCandles=", bestBearishParameters2[3],"gain-loss-out=", bestBearishParameters2[4], " resultats=",round(bestBearishGain,2) )
    except:
        print ("no bestmarameter here ")
    '''

    return {"bestBullishParameters1":bestBullishParameters1, "bestBearishParameters1":bestBearishParameters1, "bestBullishParameters2":bestBullishParameters2, "bestBearishParameters2":bestBearishParameters2, "bestBullishParameters3":bestBullishParameters3, "bestBearishParameters3":bestBearishParameters3}



Write Best parameters and traced data into files


In [ ]:
from pathlib import Path

def get_best_parameters_from_excel(symbol, frequency, period, pattern, parameter):

    workbook = openpyxl.Workbook()
    fileName = symbol.replace("/","-")+'_'+frequency+'.xlsx'
    path = "/content/gdrive/My Drive/ColabNotebooks/InvestingProject/DATA/"+str(period)+"/"
    my_file = Path(path+fileName)




    if my_file.is_file():
        workbook = openpyxl.load_workbook(my_file)
        worksheet = workbook.active

        max_row = worksheet.max_row
        max_col = worksheet.max_column
        parameters = []
        for i in range(1, max_row + 1):
            cell1 = worksheet.cell(row = i, column = 1)
            cell2 = worksheet.cell(row = i, column = 2)
            cell3 = worksheet.cell(row = i, column = 3)
            cell4 = worksheet.cell(row = i, column = 4)

            if cell1.value == symbol and  cell2.value == frequency and cell3.value == pattern and cell4.value == parameter:
                trendHeight = (worksheet.cell(row = i, column = 5)).value
                nbCandlesTrend = (worksheet.cell(row = i, column = 6)).value
                percentageCandleTrend = (worksheet.cell(row = i, column = 7)).value
                nbCandles = (worksheet.cell(row = i, column = 8) ).value
                gain = (worksheet.cell(row = i, column = 9)).value

                statBullish =  (worksheet.cell(row = i, column = 10)).value
                statBearish = (worksheet.cell(row = i, column = 11)).value
                statGain = (worksheet.cell(row = i, column = 12)).value
                statRisk = (worksheet.cell(row = i, column = 13)).value

                parameters = [nbCandlesTrend, percentageCandleTrend, trendHeight, nbCandles, gain,   statBullish, statBearish , statGain ,  statRisk]
                return parameters

    return  []

In [ ]:
def get_data_from_excel(symbol, frequency, period, pattern, parameter):

    workbook = openpyxl.Workbook()
    fileName = symbol.replace("/","-")+'_'+frequency+' data.xlsx'
    path = "/content/gdrive/My Drive/ColabNotebooks/InvestingProject/DATA/"+str(period)+"/"
    my_file = Path(path+fileName)




    if my_file.is_file():
        workbook = openpyxl.load_workbook(my_file)
        worksheet = workbook.active

        max_row = worksheet.max_row
        max_col = worksheet.max_column
        data = []
        for i in range(1, max_row + 1):
            cell1 = worksheet.cell(row = i, column = 2) #symbol
            cell2 = worksheet.cell(row = i, column = 3) #frequency
            #cell3 = worksheet.cell(row = i, column = 4)#pattern
            #cell4 = worksheet.cell(row = i, column = 5)#parameters


            if cell1.value == symbol and  cell2.value == frequency : #and cell3.value == pattern and  cell4.value == parameter:
                previousCandle = (worksheet.cell(row = i, column = 6)).value
                ConcernedCandle = (worksheet.cell(row = i, column = 7)).value
                RSI = (worksheet.cell(row = i, column = 8) ).value
                Volume = (worksheet.cell(row = i, column = 9)).value
                Risk =  (worksheet.cell(row = i, column = 11)).value
                result = (worksheet.cell(row = i, column = 12)).value


                row = {"previousCandle":previousCandle, "ConcernedCandle":ConcernedCandle, "RSI":RSI, "Volume":Volume, "Risk":Risk, "result":result}
                data.append(row)


        return data



##############################  set parameters       ####################





Detect engolfing for all symbols

In [ ]:
symbols = ['GLE.PA','CA.PA', 'RNO.PA', 'EN.PA', 'BNP.PA',  'ACA.PA', 'ORA.PA', 'VIE.PA', 'SAN.PA', 'CDI.PA', 'BIM.PA', 'DIM.PA', 'ERF.PA', 'TEP.PA', 'DSY.PA', 'HO.PA', 'SAF.PA',  'ATO.PA', 'AI.PA',  'OR.PA', 'DG.PA', 'SU.PA', 'BN.PA', 'CAP.PA',]

resultsBullish=[]
resultsBearish=[]

for symbol in symbols:

    print ("Symbol ", symbol)

    period = 2000
    end = (int)(datetime.datetime.now().timestamp())
    start =(int)((datetime.datetime.now() - datetime.timedelta(days=period)).timestamp())
    frequencies = ['1d', '1wk']



    for parameter in ["Parameter1","Parameter2"]:
        for bb in ["Bullish", "Bearish"]:
            for freq in frequencies:


                    #load data
                    df = getDataFromYahoo(symbol, start, end, freq)

                    #get parameters from excel
                    parameters = get_best_parameters_from_excel(symbol, freq, period,  bb, parameter)

                    if parameters != [] :

                          ### parameters of a trend
                          nbCandlesTrend = parameters[0] #  ###number of candles to be considered for the trend
                          percentageCandleTrend = parameters[1] # ### percentage of candles that have the same type to be considered for the trend, (for 4 candles, we have to have 3 of the same type)
                          trendHeight = parameters[2] # ###height between the first candle and the last one of the trend

                          ### parameters for checking engulfing pattern sucess
                          nbCandles = parameters[3] # ### nb candles to check the
                          gain = parameters[4] #


                          #stats
                          statBullish, statBearish , statGain ,  statRisk = parameters[5], parameters[6], parameters[7], parameters[8]

                          ### parameters for type of candle
                          percentageShadowThreshold = 0.15 ### threshold to be considered for a shaddow 15% of the total size of the candle (high - low)

                          ###add to each candle it type in the column "candleType"
                          candleSizes = getCandlesSizes(df)
                          candleBodySizes = getCandlesBodySizes(df)
                          addCandleType(df,candleSizes, candleBodySizes, percentageShadowThreshold)

                          ### add to each candle it RSI
                          ### parameters of RSI indicator
                          RSIperiod = 14
                          addRSI(df, RSIperiod)


                          #Detect patterns
                          patterns = ["bullish_engulfing", "bearish_engulfing"]
                          df = detect_pattern_engulfing(df, trendHeight, int(nbCandlesTrend), percentageCandleTrend)
                          #print (check_success_pattern_engulfing(df, nbCandles, gain, True))

                          #Alert me if the lasts candles have these paterns
                          lastCandle = df.iloc[-1]
                          oneBeforeLastCandle = df.iloc[-2]
                          #print("LastCandle : ",oneBeforeLastCandle)
                          #print (oneBeforeLastCandle[patterns[1]], oneBeforeLastCandle["datetime"])
                          if lastCandle[patterns[0]] :
                              resultsBullish.append({'symbol':symbol, 'freq':freq, 'param':parameter, "candle":lastCandle['dateTime'] , 'statBullish':statBullish, 'statGain':statGain, 'statRisk': statRisk})

                          if oneBeforeLastCandle[patterns[0]]:
                              resultsBullish.append({'symbol':symbol, 'freq':freq, 'param':parameter,  'statBullish':statBullish, 'statGain':statGain, 'statRisk': statRisk})

                              #print ("\t \t ", freq, " ", parameter, " ", patterns[0] , " -> Positive ", "statBullish:",statBullish,  "statGain:",statGain ,  "statRisk:",statRisk)

                          if lastCandle[patterns[1]] :
                              print ("yessss")
                              resultsBearish.append({'symbol':symbol, 'freq':freq, 'param':parameter, "candle":lastCandle['datetime'] , 'statBearish':statBearish, 'statGain':statGain, 'statRisk': statRisk})

                          if oneBeforeLastCandle[patterns[1]]:
                              print ("yessss ")
                              resultsBearish.append({'symbol':symbol, 'freq':freq, 'param':parameter, 'statBearishh':statBearish, 'statGain':statGain, 'statRisk': statRisk})

                              #print ("\t \t ", freq, " ", parameter, " ", patterns[1], " -> Positive ", "statBearish:",statBearish , "statGain:",statGain ,  "statRisk:",statRisk)

                              #for i in range (100):
                              #  print ("Zakaria il y a un signal sur une des deux dernières candles, regarde si c'est interessant d'investire")
                          #else :
                          #    print ("\t \t ", freq, " ", parameter, " ", bb, " -> Negative")


print ("\n \n Results Bearish")
print (resultsBearish)
print ("\n \n Results Bullish")
print (resultsBullish)


Symbol  GLE.PA
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Symbol  CA.PA
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[****